<a href="https://colab.research.google.com/github/IyadSultan/low-coding-AI/blob/main/04_tools_and_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04 Tools and Agents

A very important link https://python.langchain.com/docs/integrations/tools/

Below is a step-by-step tutorial demonstrating how to combine OpenAI Function Calling and the LangChain ecosystem for building “tools and agents.” This tutorial focuses first on the foundations—how to call functions, parse outputs, and chain calls. It then culminates in a more advanced example: reading a medical note, summarizing it, finding the best PubMed search results, retrieving references, and constructing a recommendation letter.
All code blocks are designed to run in Google Colab (or similar Python environments). Some blocks are examples to illustrate concepts (you may not need to run them all, but they’re helpful references). If you plan to save your results, be aware that Colab resets often, so consider downloading your notebook or saving to GitHub.

**1. Environment Setup**

In [1]:
import os
import json
import openai
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

**2. Quick Start with OpenAI Function Calling**

Let’s begin by showing the essential structure for function calling with the OpenAI ChatCompletion endpoints.



**3. Introducing LangChain’s “Runnable” and “Chain” Utilities**

3.1 Simple “Runnable” Pipeline
LangChain 0.0.305+ introduced the Runnable classes, which let you chain various pieces (prompts, LLMs, output parsers) in a pipeline style.


In [ ]:
!pip install langchain_openai

In [5]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
# from langchain_core.output_parsers import StrOutputParser  # Alternative import if needed
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Build a simple chain: Prompt -> LLM -> OutputParser
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)  # Specify model name
output_parser = StrOutputParser()

# Create and run the chain
chain = prompt | model | output_parser

try:
    result = chain.invoke({"topic": "bears"})
    print(result)
except Exception as e:
    print(f"An error occurred: {e}")

Why do bears have hairy coats?

Because they look silly in sweaters!


In [6]:
# Build a simple chain: Prompt -> LLM -> OutputParser
prompt = ChatPromptTemplate.from_template("How is the weather in {city}")
model = ChatOpenAI(model="gpt-4o-mini", temperature=1)  # Specify model name
output_parser = StrOutputParser()

# Create and run the chain
chain = prompt | model | output_parser

result = chain.invoke({"city": "Amman"})
print(result)


I don't have real-time data access to provide current weather updates. To find the latest weather information for Amman, I recommend checking a reliable weather website or using a weather app.


In [7]:
weather_dic={
"Amman": "Mild spring weather, around 20°C (68°F), partly cloudy",
"Amsterdam": "Cool spring conditions, around 12°C (54°F), likely overcast with chance of rain",
"Auckland": "Autumn season, mild temperatures around 18°C (64°F), partly cloudy with occasional showers",
"Athens": "Pleasant spring weather, around 22°C (72°F), mostly sunny",
"Adelaide": "Autumn weather, around 21°C (70°F), clear skies"
}

In [16]:
# prompt: write a function that uses this dictionary to return the weather in any specified city


def get_weather(city):
  """
  Returns the weather in the specified city using the weather_dic dictionary.

  Args:
      city: The name of the city.

  Returns:
      The weather description for the city, or None if the city is not found.
  """
  return weather_dic.get(city)

In [17]:
get_weather("Amman")

'Mild spring weather, around 20°C (68°F), partly cloudy'

In [21]:

get_weather_json_format={
        "name": "get_weather",
        "description": "Get the current weather for a given city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The name of the city to get weather for."
                }
            },
            "required": ["city"]
        }
    }


# Define the function as a tool
functions = [get_weather_json_format]


from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)


def chat_with_gpt(user_message):
    """
    Sends a message to the GPT model and allows it to use the get_weather function when needed.
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Correct model name
        messages=[{"role": "user", "content": user_message}],
        functions=functions,
        function_call="auto"  # Enable function calling
    )


    message = response.choices[0].message


    # Handle function calls if present
    if message.function_call:
        function_name = message.function_call.name
        function_args = json.loads(message.function_call.arguments)

        if function_name == "get_weather":
            function_response = get_weather(function_args["city"])

            # Send the function response back to GPT
            final_response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "user", "content": user_message},
                    message,
                    {
                        "role": "function",
                        "name": "get_weather",
                        "content": function_response
                    }
                ]
            )
            return final_response.choices[0].message.content

    return message.content


# Example Usage
user_input = "What's the weather like in Amman today?"
print(chat_with_gpt(user_input))


The weather in Amman today is mild spring weather, with temperatures around 20°C (68°F) and partly cloudy skies.


In [23]:
def tip(x):
  return x*0.15



15.0

In [10]:
# prompt: make a pydanti model that use the tip funciton

from pydantic import BaseModel, Field

class TipCalculator(BaseModel):
    amount: float = Field(..., description="The amount to calculate the tip on.")

    def tip(self):
      return self.amount * 0.15

In [26]:
TipCalculator(amount=100).tip()

15.0

In [24]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

from langchain.utils.openai_functions import convert_pydantic_to_openai_function
import json
from openai import OpenAI
from pprint import pprint

tips=convert_pydantic_to_openai_function(TipCalculator)
# pprint(tips)

client = OpenAI(api_key=OPENAI_API_KEY)

user_message = "Our lunch bill was 80 JD, how much should I tip the waiter?"

response=client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": user_message},
    ],
    functions=[tips],
    function_call="auto"
)

pprint(response)




ChatCompletion(id='chatcmpl-AvSkGxlciRHyRxYWXAgeyqsWk7AAw', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"amount":80}', name='TipCalculator'), tool_calls=None))], created=1738258556, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_72ed7ab54c', usage=CompletionUsage(completion_tokens=16, prompt_tokens=62, total_tokens=78, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [25]:
# prompt: extract function_call from this if it exists and execute the function using the arguement
# ChatCompletion(id='chatcmpl-AvSkGxlciRHyRxYWXAgeyqsWk7AAw', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"amount":80}', name='TipCalculator'), tool_calls=None))], created=1738258556, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_72ed7ab54c', usage=CompletionUsage(completion_tokens=16, prompt_tokens=62, total_tokens=78, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

import json

# Assuming 'response' is defined as in the provided code
function_call = response.choices[0].message.function_call

if function_call:
    function_name = function_call.name
    arguments = json.loads(function_call.arguments)

    if function_name == "TipCalculator":
        tip_calculator = TipCalculator(**arguments)
        tip_amount = tip_calculator.tip()
        print(f"The tip amount is: {tip_amount}")



The tip amount is: 12.0


In [38]:
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from typing import Optional
from datetime import datetime

# Create a mock weather database
weather_db = {
    "SFO": {
        "city": "San Francisco",
        "temperature": 65,
        "conditions": "Foggy",
        "humidity": 75,
        "wind_speed": 12,
        "last_updated": "2025-01-29 08:00:00"
    },
    "JFK": {
        "city": "New York",
        "temperature": 45,
        "conditions": "Partly Cloudy",
        "humidity": 60,
        "wind_speed": 15,
        "last_updated": "2025-01-29 08:00:00"
    },
    "LAX": {
        "city": "Los Angeles",
        "temperature": 75,
        "conditions": "Sunny",
        "humidity": 50,
        "wind_speed": 8,
        "last_updated": "2025-01-29 08:00:00"
    },
    "ORD": {
        "city": "Chicago",
        "temperature": 32,
        "conditions": "Snow",
        "humidity": 80,
        "wind_speed": 20,
        "last_updated": "2025-01-29 08:00:00"
    },
    "AMM": {
        "city": "Amman",
        "temperature": 82,
        "conditions": "Thunderstorms",
        "humidity": 85,
        "wind_speed": 18,
        "last_updated": "2025-01-29 08:00:00"
    }
}

class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

    def get_weather(self) -> Optional[dict]:
        """Get weather for the specified airport"""
        try:
            if self.airport_code not in weather_db:
                return {
                    "error": f"No weather data available for {self.airport_code}",
                    "available_airports": list(weather_db.keys())
                }

            weather_data = weather_db[self.airport_code]
            return {
                "airport": self.airport_code,
                "city": weather_data["city"],
                "temperature": f"{weather_data['temperature']}°F",
                "conditions": weather_data["conditions"],
                "humidity": f"{weather_data['humidity']}%",
                "wind_speed": f"{weather_data['wind_speed']} mph",
                "last_updated": weather_data["last_updated"]
            }
        except Exception as e:
            return {"error": str(e)}

weather_function = convert_pydantic_to_openai_function(WeatherSearch)

# Test the function with different airports
test_codes = ["QAA", "SFO", "JFK", "LAX", "ORD", "DFW"]  # DFW isn't in our database

for code in test_codes:
    try:
        search = WeatherSearch(airport_code=code)
        weather_data = search.get_weather()
        print(f"\nWeather lookup for {code}:")
        print(weather_data)
    except Exception as e:
        print(f"\nError processing {code}: {e}")

# Print the function definition
print("\nFunction definition:")
print(weather_function)


Weather lookup for QAA:
{'error': 'No weather data available for QAA', 'available_airports': ['SFO', 'JFK', 'LAX', 'ORD', 'AMM']}

Weather lookup for SFO:
{'airport': 'SFO', 'city': 'San Francisco', 'temperature': '65°F', 'conditions': 'Foggy', 'humidity': '75%', 'wind_speed': '12 mph', 'last_updated': '2025-01-29 08:00:00'}

Weather lookup for JFK:
{'airport': 'JFK', 'city': 'New York', 'temperature': '45°F', 'conditions': 'Partly Cloudy', 'humidity': '60%', 'wind_speed': '15 mph', 'last_updated': '2025-01-29 08:00:00'}

Weather lookup for LAX:
{'airport': 'LAX', 'city': 'Los Angeles', 'temperature': '75°F', 'conditions': 'Sunny', 'humidity': '50%', 'wind_speed': '8 mph', 'last_updated': '2025-01-29 08:00:00'}

Weather lookup for ORD:
{'airport': 'ORD', 'city': 'Chicago', 'temperature': '32°F', 'conditions': 'Snow', 'humidity': '80%', 'wind_speed': '20 mph', 'last_updated': '2025-01-29 08:00:00'}

Weather lookup for DFW:
{'error': 'No weather data available for DFW', 'available_airpo

In [ ]:
print(weather_function)

**4.2 Binding Functions to a Chat Model**

In [75]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5, api_key=OPENAI_API_KEY).bind(functions=[weather_function])

def process_weather_query(query):
    response = model.invoke(query)
    if 'function_call' in response.additional_kwargs:
        args = json.loads(response.additional_kwargs['function_call']['arguments'])
        return WeatherSearch(**args).get_weather()
    return "Weather data unavailable"

# Create prompt template and chain once
prompt = ChatPromptTemplate([("human", "Nicely describe the weather in this airport and wish a nice trip using this data: {weather_data}")])
chain = prompt | model | StrOutputParser()

# Process queries
for query in ["What's the weather in SFO?", "Weather at AMMAN", "What is the weather in CCC"]:
    weather_data = process_weather_query(query)
    # print(f"\nQuery: {query}")
    # print(f"Weather data: {weather_data}")
    # Pass the weather data with the correct variable name
    response = chain.invoke({"weather_data": weather_data})
    print(f"Description: {response}")

Description: The weather at San Francisco International Airport (SFO) is currently a cool 65°F, enveloped in a gentle fog that adds a touch of mystique to the surroundings. With a humidity level of 75%, the air feels refreshingly crisp, complemented by a light breeze blowing at 12 mph. 

As you prepare for your journey, embrace the enchanting atmosphere of San Francisco. Wishing you a wonderful trip filled with delightful experiences and safe travels!
Description: The weather at Queen Alia International Airport (AMM) in Amman is currently quite dynamic, with a temperature of 82°F. Thunderstorms are rolling through the area, creating a dramatic backdrop for your travels. The humidity is elevated at 85%, adding a touch of warmth to the air, while a brisk wind is blowing at 18 mph, offering a refreshing contrast.

As you prepare for your journey, please stay safe and keep an eye on the weather updates. Wishing you a wonderful trip filled with memorable experiences!
Description: It seems t

**4.3 Force-Calling a Function**
You can also force the model to always call a particular function:


In [ ]:
model_with_function = model.bind(
    functions=[weather_function],
    function_call={"name": "WeatherSearch"}  # Force the model to always use this function
)
output_parser = StrOutputParser()
chain=model_with_function|output_parser
# print(model_with_function.invoke("What's the weather in SFO?"))
chain.invoke("What's the weather in SFO?")


**5. Tools and Agents in LangChain**

In LangChain, “tools” are simply Python callables (functions) that can be exposed to an LLM. Agents decide which tool to call and when to call it.
5.1 Defining Tools
LangChain supplies a @tool decorator for convenience. Example:
from langchain.agents import tool


In [ ]:
# !pip install Wikipedia
from langchain.agents import tool
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Search Wikipedia and get page summaries."""
    try:
        # Get search results
        page = wikipedia.page(query, auto_suggest=False)
        return f"Title: {page.title}\nSummary: {page.summary}"
    except:
        try:
            # If direct search fails, try getting the first result from search
            results = wikipedia.search(query, results=1)
            if results:
                page = wikipedia.page(results[0], auto_suggest=False)
                return f"Title: {page.title}\nSummary: {page.summary}"
        except:
            pass

    return "No Wikipedia article found."

print(search_wikipedia("Python programming"))


**5.2 Converting Tools to OpenAI Functions**

In [ ]:
from langchain.tools.render import format_tool_to_openai_function

formatted_tool = format_tool_to_openai_function(search_wikipedia)
print(formatted_tool)


**5.3 A Simple Routing Example**
If you pass multiple tools, the LLM will pick which function to call or produce a direct textual response. For instance:



In [ ]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function


functions = [formatted_tool, weather_function]  # Suppose we have two


model = ChatOpenAI(temperature=0).bind(functions=functions)
chain=model|output_parser
chain.invoke("What's the weather in SFO?")
# result = model.invoke("What is the weather in Boston right now?")
# print(result)

In [ ]:
%pip install xmltodict

In [ ]:
from langchain_community.tools.pubmed.tool import PubmedQueryRun

# Initialize the PubMed query tool
pubmed_tool = PubmedQueryRun()

# Define the search query for lung cancer
query = "lung cancer"

# Run the query to get results
results = pubmed_tool.invoke(query)

# Extract and display 10 abstracts from the results
print(results)

In [ ]:
from langchain_community.retrievers import PubMedRetriever
retriever = PubMedRetriever()
result=retriever.invoke('Lung Cancer')
print(len(result))
# print(result[0])


In [ ]:
!pip install langchain_google_community

from langchain_core.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

**6. Building a Conversational Agent with Memory**
Let’s combine:
Tools for weather, Wikipedia.
A Chat model that can call them as needed.
Conversation memory so the agent can remember earlier turns.


In [ ]:
# !pip install langchain_community
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain.agents import tool
from langchain_openai import ChatOpenAI
from langchain.tools.render import format_tool_to_openai_function
from langchain.schema.runnable import RunnablePassthrough
import wikipedia


@tool
def search_wikipedia(query: str) -> str:
    """Search Wikipedia and get page summaries."""
    try:
        page = wikipedia.page(query, auto_suggest=False)
        return f"Title: {page.title}\nSummary: {page.summary}"
    except:
        try:
            results = wikipedia.search(query, results=1)
            if results:
                page = wikipedia.page(results[0], auto_suggest=False)
                return f"Title: {page.title}\nSummary: {page.summary}"
        except:
            pass
    return "No Wikipedia article found."

@tool
def get_current_temperature(location: str) -> str:
    """Get the current temperature for a location."""
    return f"It is 70F in {location}"

# Helper function to format intermediate steps
def format_to_openai_functions(intermediate_steps):
    """Format intermediate steps to OpenAI function messages."""
    messages = []
    for action, observation in intermediate_steps:
        messages.append({
            "tool": action.tool,
            "tool_input": action.tool_input,
            "observation": observation
        })
    return messages

# Format tools
tools = [search_wikipedia, get_current_temperature]
functions = [format_tool_to_openai_function(t) for t in tools]

# Initialize model with functions
model = ChatOpenAI(temperature=0).bind(functions=functions)

# Create prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Remember information about the user."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Create the chain with initial empty intermediate steps
chain = RunnablePassthrough.assign(
    chat_history=lambda x: x.get("chat_history", []),
    intermediate_steps=lambda x: x.get("intermediate_steps", []),
    agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

# Initialize memory
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

# Create agent executor
agent_executor = AgentExecutor(
    agent=chain,
    tools=tools,
    verbose=True,
    memory=memory
)

# Test the agent
try:
    # First interaction
    response1 = agent_executor.invoke({"input": "My name is Bob."})
    print("Response 1:", response1)

    # Second interaction
    response2 = agent_executor.invoke({"input": "What's my name?"})
    print("Response 2:", response2)
except Exception as e:
    print(f"An error occurred: {str(e)}")